In [2]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

import plotly.express as px

import xgboost as xgb

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [5]:
stats = pd.read_csv('../pipeline_season_testing/data/07_seasontest_adjusted_value_models_aggregated.csv')
schedule = pd.read_csv('../data/schedule_final.csv')
betting = pd.read_csv('../data/betting_data_cleaned_with_returns.csv')
betting = betting.drop(columns=['Unnamed: 0'])
current_week = pd.read_csv('../pipeline_season_testing/data/04_seasontest_current_week_data.csv')

stats = stats.dropna()

In [14]:
stats

,Unnamed: 0,season,week,team,qb,passing_value_adjusted,rushing_value_adjusted,total_plays_standardized,total_possession_time_standardized,pass_percentage_standardized,pass_def_value_adjusted,rush_def_value_adjusted,special_teams_value,team_full
64,64,2022,3,ARI,K. Murray,-0.484359,1.210528,0.756386,0.345579,0.684827,0.113894,0.094269,-0.350135,Arizona Cardinals
65,65,2022,3,ATL,M. Mariota,-0.299158,-0.536694,0.385318,0.642629,-0.671472,-0.246205,0.027116,0.802206,Atlanta Falcons
66,66,2022,3,BAL,L. Jackson,1.294969,-0.195054,-1.261296,-0.999246,-0.008838,-0.991046,-0.509766,1.363465,Baltimore Ravens
67,67,2022,3,BUF,J. Allen,0.941679,-0.105653,0.014250,0.471343,0.425350,1.172158,0.561945,1.339333,Buffalo Bills
68,68,2022,3,CAR,B. Mayfield,-0.781955,1.061613,-1.261296,-1.710622,0.207161,0.018787,0.814432,-1.336395,Carolina Panthers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,562,2022,21,PHI,J. Hurts,0.218471,1.429204,0.744790,0.553514,-0.749986,0.738168,-0.028500,0.353784,Philadelphia Eagles
564,564,2022,22,KC,P. Mahomes,0.596992,0.105791,-0.142294,0.056631,0.469324,0.570522,-0.627846,-0.488477,Kansas City Chiefs
565,565,2022,22,PHI,J. Hurts,0.053270,1.144668,0.767982,0.920775,-1.140699,0.531111,-0.066150,0.098544,Philadelphia Eagles
566,566,2023,1,KC,P. Mahomes,1.000860,0.727819,-0.426393,-0.357117,0.117055,-0.066309,-0.265523,-0.380864,Kansas City Chiefs


In [12]:
current_week

,Unnamed: 0,season,week,away,home,away_moneyline,home_moneyline,away_spread,over_under,home_full_name,away_full_name,home_qb,away_qb
0,202311,2023,1,DET,KC,-225,185,4.5,52.5,Kansas City Chiefs,Detroit Lions,P. Mahomes,J. Goff


In [15]:
schedule

,Unnamed: 0,date,season,week,season_type,home,away,home_score,away_score,home_qb,away_qb,home_qb_abv,away_qb_abv
0,0,2014-09-04,2014,1,REG,Seattle Seahawks,Green Bay Packers,36,16,Russell Wilson,Aaron Rodgers,R. Wilson,A. Rodgers
1,1,2014-09-07,2014,1,REG,Baltimore Ravens,Cincinnati Bengals,16,23,Joe Flacco,Andy Dalton,J. Flacco,A. Dalton
2,2,2014-09-07,2014,1,REG,Houston Texans,Washington Football Team,17,6,Ryan Fitzpatrick,Robert Griffin,R. Fitzpatrick,R. Griffin
3,3,2014-09-07,2014,1,REG,Chicago Bears,Buffalo Bills,20,23,Jay Cutler,EJ Manuel,J. Cutler,E. Manuel
4,4,2014-09-07,2014,1,REG,Pittsburgh Steelers,Cleveland Browns,30,27,Ben Roethlisberger,Brian Hoyer,B. Roethlisberger,B. Hoyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2433,2433,2023-01-22,2022,20,POST,Buffalo Bills,Cincinnati Bengals,10,27,Josh Allen,Joe Burrow,J. Allen,J. Burrow
2434,2434,2023-01-22,2022,20,POST,San Francisco 49ers,Dallas Cowboys,19,12,Brock Purdy,Dak Prescott,B. Purdy,D. Prescott
2435,2435,2023-01-29,2022,21,POST,Philadelphia Eagles,San Francisco 49ers,31,7,Jalen Hurts,Brock Purdy,J. Hurts,B. Purdy
2436,2436,2023-01-29,2022,21,POST,Kansas City Chiefs,Cincinnati Bengals,23,20,Patrick Mahomes,Joe Burrow,P. Mahomes,J. Burrow


In [23]:
nfl = current_week.copy()
nfl = nfl.merge(stats.add_suffix('_home'), how='left', left_on = ['season', 'week', 'home_full_name', 'home_qb'], 
                right_on = ['season_home', 'week_home', 'team_full_home', 'qb_home'])
nfl = nfl.merge(stats.add_suffix('_away'), how='left', left_on = ['season', 'week', 'away_full_name', 'away_qb'], 
                right_on = ['season_away', 'week_away', 'team_full_away', 'qb_away'])
nfl['home_spread'] = -nfl['away_spread']
nfl['total_score_line'] = nfl['over_under']
nfl = nfl[['season', 'week','home', 'away', 'home_qb', 'away_qb',
          'passing_value_adjusted_home', 'rushing_value_adjusted_home', 'pass_def_value_adjusted_home', 'rush_def_value_adjusted_home', 'special_teams_value_home',
          'passing_value_adjusted_away', 'rushing_value_adjusted_away', 'pass_def_value_adjusted_away', 'rush_def_value_adjusted_away', 'special_teams_value_away',
          'total_possession_time_standardized_home', 'total_possession_time_standardized_away', 
          'total_plays_standardized_home', 'total_plays_standardized_away',
          'pass_percentage_standardized_home', 'pass_percentage_standardized_away',
          'home_moneyline', 'away_moneyline', 'home_spread', 'away_spread', 'total_score_line']]
nfl.head()

,season,week,home,away,home_qb,away_qb,passing_value_adjusted_home,rushing_value_adjusted_home,pass_def_value_adjusted_home,rush_def_value_adjusted_home,special_teams_value_home,passing_value_adjusted_away,rushing_value_adjusted_away,pass_def_value_adjusted_away,rush_def_value_adjusted_away,special_teams_value_away,total_possession_time_standardized_home,total_possession_time_standardized_away,total_plays_standardized_home,total_plays_standardized_away,pass_percentage_standardized_home,pass_percentage_standardized_away,home_moneyline,away_moneyline,home_spread,away_spread,total_score_line
0,2023,1,KC,DET,P. Mahomes,J. Goff,1.00086,0.727819,-0.066309,-0.265523,-0.380864,0.871925,-0.399306,0.720782,-0.453181,-0.181762,-0.357117,-0.054666,-0.426393,0.159198,0.117055,0.05032,185,-225,-4.5,4.5,52.5
